# 2024 OIBC Challenge
## (Deep Learning version skeleton 02)

### 데이터 전처리

In [1]:
import pandas as pd
from functools import reduce
import os
print(os.getcwd())  # 현재 작업 경로 출력

f:\For me\Project\24-OIBC-WolgoksTears\personal_files\hoon


actual_weather_1

In [2]:
# 데이터 불러오기
actual_weather_1 = pd.read_csv("../../data_files/actual_weather_1.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
actual_weather_1['datetime'] = pd.to_datetime(actual_weather_1['ts'], unit='s')
actual_weather_1['datetime'] = actual_weather_1['datetime'].dt.floor('h')

# 원래 'ts' 열과 'location' 제거
actual_weather_1.drop(columns=['ts'], inplace=True)
actual_weather_1.drop(columns=['location'], inplace=True)

# 1시간 단위로 그룹화하여 평균을 계산 (location을 제외한 모든 열)
actual_weather_1 = actual_weather_1.groupby(['datetime'], as_index=False).mean()


# weather_forecast에 없는 columns 제거
actual_weather_1.drop(columns=['real_feel_temp_shade'], inplace=True)
actual_weather_1.drop(columns=['pressure'], inplace=True)
actual_weather_1.drop(columns=['appr_temp'], inplace=True)
actual_weather_1.drop(columns=['wind_chill_temp'], inplace=True)
 
# 결과 확인
actual_weather_1.head()

,datetime,temp,real_feel_temp,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,vis,cld_cvr,ceiling,wet_bulb_temp,precip_1h
0,2024-02-29 15:00:00,7.140000,3.490000,73.400000,2.650000,254.500000,18.310000,24.020000,0.0,18.020000,97.500000,697.900000,5.150000,0.020000
1,2024-02-29 16:00:00,7.700000,3.400000,71.000000,2.825000,219.750000,21.375000,27.625000,0.0,26.550000,96.000000,708.750000,5.475000,0.000000
2,2024-02-29 17:00:00,6.105882,1.164706,75.235294,1.894118,235.882353,23.005882,28.288235,0.0,20.264706,74.000000,5685.352941,4.329412,0.076471
3,2024-02-29 18:00:00,5.618182,0.681818,69.818182,0.436364,241.727273,22.927273,37.345455,0.0,20.609091,43.909091,9304.727273,3.472727,0.400000
4,2024-02-29 19:00:00,4.770000,-0.690000,75.100000,0.540000,227.500000,25.780000,35.140000,0.0,21.890000,22.900000,11399.500000,3.090000,0.000000


weather_forecast

In [3]:
# 데이터 불러오기
weather_forecast_1 = pd.read_csv("../../data_files/weather_forecast_1.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
weather_forecast_1['datetime'] = pd.to_datetime(weather_forecast_1['ts'], unit='s')
weather_forecast_1['datetime'] = weather_forecast_1['datetime'].dt.floor('h')

# 원래 'ts' 열 제거, 'base_ts' 열 제거, 'location' 제거
weather_forecast_1.drop(columns=['ts'], inplace=True)
weather_forecast_1.drop(columns=['base_ts'], inplace=True)
weather_forecast_1.drop(columns=['location'], inplace=True)

# 1시간 단위로 그룹화하여 평균을 계산 (location을 제외한 모든 열)
weather_forecast_1 = weather_forecast_1.groupby(['datetime'], as_index=False).mean()


# 예보 데이터의 columns 순서를 actual_weather columns 순서와 맞추기
weather_forecast_1 = weather_forecast_1[['datetime', 'temp', 'real_feel_temp', 'rel_hum',
       'dew_point', 'wind_dir', 'wind_spd', 'wind_gust_spd', 'uv_idx', 'vis',
       'cld_cvr', 'ceiling', 'wet_bulb_temp', 'total_liq']]
weather_forecast_1['precip_1h'] = weather_forecast_1['total_liq']
weather_forecast_1.drop(columns=['total_liq'], inplace=True)  

# 결과 확인
weather_forecast_1.head()

,datetime,temp,real_feel_temp,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,vis,cld_cvr,ceiling,wet_bulb_temp,precip_1h
0,2024-03-01 15:00:00,1.728394,-4.320987,64.333333,-4.320987,335.555556,22.048022,50.604944,0.0,16.093400,75.555556,1700.106667,-0.555556,0.000000
1,2024-03-01 16:00:00,1.851852,-4.506173,64.333333,-4.135800,335.000000,22.048022,50.640711,0.0,15.378140,84.777778,1249.680000,-0.432099,0.056444
2,2024-03-01 17:00:00,1.790123,-5.061728,66.333333,-3.950616,334.444444,22.656000,51.481144,0.0,14.392882,89.333333,1229.360000,-0.370370,0.197556
3,2024-03-01 18:00:00,1.604940,-5.308640,67.333333,-3.888887,333.777778,23.263967,51.463267,0.0,15.195758,94.555556,1933.786667,-0.370370,0.084667
4,2024-03-01 19:00:00,1.851852,-4.876541,66.444444,-3.950616,333.888889,23.871922,50.837400,0.0,15.825200,82.555556,2600.960000,-0.308642,0.000000


elec_supply

In [4]:
# 데이터 불러오기
elec_supply = pd.read_csv("../../data_files/elec_supply.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
elec_supply['datetime'] = pd.to_datetime(elec_supply['ts'], unit='s')
elec_supply['datetime'] = elec_supply['datetime'].dt.floor('h')

# 1시간 단위로 그룹화하여 평균을 계산
elec_supply = elec_supply.groupby(['datetime'], as_index=False).mean()

# 결과 확인
elec_supply.drop(columns=['ts'], inplace=True)  # 원래 'ts' 열 제거
elec_supply.head()

,datetime,공급능력(kW),현재 수요(kW),태양광 발전량kW),풍력 발전량(kW),신재생 발전량 총합(kW),공급 예비력(kW),운영 예비력(kW)
0,2023-12-31 15:00:00,1.266917e+06,748333.333333,0.0,97205.025000,105234.625000,518416.666667,320166.666667
1,2023-12-31 16:00:00,1.223083e+06,711666.666667,0.0,54451.566667,62453.733333,511416.666667,313750.000000
2,2023-12-31 17:00:00,1.214917e+06,681500.000000,0.0,45310.175000,53249.933333,533333.333333,307333.333333
3,2023-12-31 18:00:00,1.198250e+06,666916.666667,0.0,29522.183333,37464.758333,531500.000000,297666.666667
4,2023-12-31 19:00:00,1.193500e+06,666833.333333,0.0,22640.991667,30842.566667,526750.000000,311833.333333


smp_da

In [5]:
# 데이터 불러오기
smp_da = pd.read_csv("../../data_files/smp_da.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
smp_da['datetime'] = pd.to_datetime(smp_da['ts'], unit='s')

# 결과 확인
smp_da.drop(columns=['ts'], inplace=True)  # 원래 'ts' 열 제거
smp_da.head()

,하루전가격(원/kWh),datetime
0,107.39,2024-02-29 15:00:00
1,107.39,2024-02-29 16:00:00
2,95.30,2024-02-29 17:00:00
3,87.89,2024-02-29 18:00:00
4,0.00,2024-02-29 19:00:00


smp_rt_rc

In [6]:
# 데이터 불러오기
smp_rt_rc = pd.read_csv("../../data_files/smp_rt_rc.csv")

# 'ts'를 datetime 형식으로 변환 후 1시간 단위로 맞추기
smp_rt_rc['datetime'] = pd.to_datetime(smp_rt_rc['ts'], unit='s')
smp_rt_rc['datetime'] = smp_rt_rc['datetime'].dt.floor('h')

# 1시간 단위로 그룹화하여 평균을 계산
smp_rt_rc = smp_rt_rc.groupby(['datetime'], as_index=False).mean()

# 결과 확인
smp_rt_rc.drop(columns=['ts'], inplace=True)  # 원래 'ts' 열 제거
smp_rt_rc.head()

,datetime,실시간 임시 가격(원/kWh),실시간 확정 가격(원/kWh)
0,2024-02-29 15:00:00,136.0150,136.0150
1,2024-02-29 16:00:00,98.3225,98.3225
2,2024-02-29 17:00:00,89.7425,89.7425
3,2024-02-29 18:00:00,86.8475,86.8475
4,2024-02-29 19:00:00,88.2100,88.2100


##### X 데이터

- 제주 기상 실측 (n+1)
- 제주 전력 시장 실시간 (n-1)
- 제주 전력 시장 현황 (n-1)

In [ ]:
actual_weather_1
smp_rt_rc['datetime'] += pd.Timedelta(days=2)
elec_supply['datetime'] += pd.Timedelta(days=2)

In [28]:
dfs = [actual_weather_1, smp_rt_rc, elec_supply]

X = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='inner'), dfs)

##### Y 데이터

- 하루 전 시장 전기 예측 가격(n+1)

In [29]:
Y = smp_da

### 파이토치 모델 구성

##### 날짜 설정

In [30]:
target_date = {
  # 최소 2024-03-03
  "train_start_date": '2024-03-03',
  # 최대 2024-10-22
  "train_end_date": '2024-10-22',
  # 예측할 날짜
  "predict_target_date": '2024-11-03',
}

In [ ]:
X = X.loc[
  (X['datetime'] > target_date['train_start_date']) &
  (X['datetime'] <= target_date['train_end_date'])
]

In [34]:
Y = Y.loc[
  (Y['datetime'] > target_date['train_start_date']) &
  (Y['datetime'] <= target_date['train_end_date'])
]

##### 모듈 임포트

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler
import datetime

In [19]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_name())
  print(torch.__version__)
  print(torch.version.cuda)
  x = torch.randn(1).cuda()
  print(x)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

NVIDIA GeForce GTX 1650
2.5.0+cu118
11.8
tensor([-1.5809], device='cuda:0')


device(type='cuda', index=0)

##### Dataset 구성

In [50]:
X.drop(columns=['datetime']).iloc[0].to_numpy()

array([ 8.49000000e+00,  5.68000000e+00,  5.42000000e+01, -3.70000000e-01,
        2.58900000e+02,  1.75500000e+01,  1.92200000e+01,  1.40000000e+00,
        1.85000000e+01,  9.72000000e+01,  3.59070000e+03,  4.64000000e+00,
        1.60000000e-01,  1.23925000e+01,  1.23925000e+01,  1.53608333e+06,
        7.21833333e+05,  1.56524417e+05,  2.22763417e+05,  3.93779333e+05,
        8.14166667e+05,  4.55583333e+05])

In [ ]:
class ElecDataset(Dataset):
  def __init__(self, x_data, y_data):
    scaler = MinMaxScaler()

    columns_to_scale = x_data.columns[1:]
    x_data[columns_to_scale] = scaler.fit_transform(x_data[columns_to_scale])
    
    self.x_data = x_data
    self.y_data = y_data

  def __getitem__(self, index):
    target_y = self.y_data['하루전가격(원/kWh)'].iloc[index]
    targets = self.x_data.drop('datetime').iloc[index].to_numpy()

    return torch.from_numpy(targets), torch.tensor(target_y)

  def __len__(self):
    return len(self.y_data)

In [52]:
dataset = ElecDataset(X, Y)

##### dataloader 구성

In [ ]:
dataloader = DataLoader(dataset=dataset,
                        batch_size=24,
                        # 데이터의 순서가 중요해서 shuffle하면 안될 듯
                        shuffle=False,
                        drop_last=False)

##### 모델 구성

In [ ]:
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
    super(LSTMModel, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    
    # Define the LSTM layer
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dtype=torch.float64)
    
    # Define a fully connected layer to produce output of size 1
    self.fc = nn.Linear(hidden_size, output_size, dtype=torch.float64)
  
  def forward(self, x):
    # Set initial hidden and cell states to zeros
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float64).to(x.device)  # hidden state
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, dtype=torch.float64).to(x.device)  # cell state
    
    # Forward propagate LSTM
    out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
    
    # Get the last time step output (last hidden state)
    out = out[:, -1, :]  # (batch_size, hidden_size)
    
    # Pass the output through the fully connected layer
    out = self.fc(out)  # (batch_size, 1)
    
    return out